# STA663 Final Project: Stochstic Gradient Hamiltonian Monte Carlo in Python

## 1.Introduction and Background

Hamiltonian Monte Carlo(HMC) sampling methods define a target function by learning from the transformation between potential and kinetic energy and parameterize them by introducing 'momentum' auxiliary variables. Under Hammiltonian dynamics, the target distribution is invariant and the use of Metropolis-Hastings samplings achieves high acceptance rate. However, one limitation of HMC is its computational complexity increasing with data size due to the necessity of computing potential energy gradient for each simulaiton. Therefore, in order to make HMC algorithm more applicable to large dataset problem, Chen.T, Fox.E and Guestrin.C, introduced Stochastic Gradient Hamiltonian Monte Carlo(SGHMC) sampling method in 2014.

SGHMC maintains the stationarity of the target distribution as HMC and 

We attempt to improve HMC in an orthogonal direction focused on computational complexity

## 2. Implementation

### 2.1 Algorithm

#### Hamiltonian Monte Carlo (HMC）

In order to sample from the posterior distribution $p(\theta|D)$, HMC introduces a set of auxiliary momentum variables $\textit{r}$ to establish a new joint distribution $(\theta, r)$:
$$
\pi(\theta,r)\propto \exp(-U(\theta)-\frac{1}{2}r^TM^{-1}r)\ \ \  (1)
$$
where $\textit{U}$ is a potential energy function,
$$
U = \sum_{x\in D}log p(x|D)-log p(\theta)\ \ \ (2)
$$
and $\theta$ can be sampled from the marginal distribution $p(\theta|D)$.

Learning from a physical definition, Hamiltonian function: $H(\theta,r)= U(\theta)+\frac{1}{2}r^TM^{-1}r$,can be seen as the measurement of the total energy of a physical system and the dynamic equations are the following:
$$
d\theta = M^{-1}r dt \ \ (3)
$$
$$
dr = - \nabla U(\theta) dt \ \  (4)
$$

#### Stochastic Gradient HMC (SGHMC)

The improvement made by stochastic gradient HMC is instead of computing $\nabla U$ using equation (4) in the whole dataset, SGHMC computes a $\tilde U$ on a minibacth $\tilde D$ uniformly sampled from the entire dataset D:
$$
\nabla \tilde U(\theta)=-\frac{|D|}{|\tilde D|}\sum_{x\in \tilde{D}}\nabla logp(x|\theta)-\nabla logp(\theta), \tilde{D} \in D \ \ \ (5)
$$

which can be approximated by:
$$
\nabla \tilde U(\theta) \simeq \nabla U(\theta)+N(0,V(\theta))\ \ \ (6)
$$

Therefore, replace $\nabla U(\theta)$ in (4) by $\nabla \tilde U(\theta)$ in (6) and the dynamic equations become:
$$
d \theta = M^{-1}rdt\ \ \ (7)
$$
$$
dr = -\nabla U(\theta) + N(0,2B(\theta)dt)\ \ \ (8)
$$

where $B(\theta)=\frac{1}{2}\epsilon V(\theta)$, and $\epsilon$ is a step term introduced from Metroplis-Hasting sampling, since above dynamic equations are considered as discretized systems in practice, so MH steps must be implemented. 

Nonetheless, $\pi(\theta,r)$ is not invariant under the new dynamics of equations (7) and (8), because of $B(\theta)$ being a positive semi-definite matrix contributed by gradient noise, a $\textit{Friction}$ term is suggested to be added by Chen et al.(2014). This modification takes the edge of the noises caused by stochastic gradient and make the target distribution $\pi(\theta,r)$ invariant again. Therefore, the dynamic equations become the following:
$$
d \theta = M^{-1}rdt\ \ \ \ (9)
$$
$$
dr = -\nabla U(\theta) dt - BM^{-1}rdt+N(0,2B(\theta)dt) \ \ \ \ (10)
$$

#### Stochastic Gradient HMC (SGHMC) in Practice

As mentioned above, B is the noise matrix and is estimated as $\hat{B}$ in practice with a user specified friction term $C$, where $C \geq \hat{B}$. Therefore, the dynamic equations become:
$$
d \theta = M^{-1}rdt\ \ \ (11)
$$
$$
dr = -\nabla U(\theta) dt - CM^{-1}rdt+N(0,2(C-\hat{B})dt)+N(0,2B(\theta)dt) \ \ \ \ (12)
$$

The true stochastic gradient noise matrix B is not zero as $B=\frac{1}{2}\epsilon V$, but if the step size $\epsilon$ tends to zero, B becomes zero and the user specified $C$ dominants.

Therefore, this algorithm becomes:
Take the initial values of $(\theta_0, r_0,\epsilon, M,\hat{B},B,C)$, then updata through：

$$
\theta_i=\theta_{i-1}+\epsilon_tM^{-1}r_{i-1}
$$
$$
r_i=r_{i-1}-\epsilon_t \nabla \tilde {U}(\theta_i)-\epsilon_t CM^{-1}r_{i-1}+N(0,2(C-\hat{B})\epsilon_t)
$$
When i = 0,1,2,...